In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import re, string
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
print('Load train data, test data and submission file')
train = pd.read_csv('train_clean.csv', low_memory=False, encoding='latin1')
final_out=pd.read_csv('Devex_submission_format.csv', low_memory=False, encoding='latin1')
test = pd.read_csv("Devex_test_questions.csv",low_memory=False, encoding='latin1')

In [ ]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [ ]:
COMMENT = 'Text'
train[COMMENT].fillna("unknown", inplace=True)
#test[COMMENT].fillna("unknown", inplace=True)


In [ ]:
label_cols = [
 '3.1.1',
 '3.1.2',
 '3.2.1',
 '3.2.2',
 '3.3.1',
 '3.3.2',
 '3.3.3',
 '3.3.4',
 '3.3.5',
 '3.4.1',
 '3.4.2',
 '3.5.1',
 '3.5.2',
 '3.6.1',
 '3.7.1',
 '3.7.2',
 '3.8.1',
 '3.8.2',
 '3.9.1',
 '3.9.2',
 '3.9.3',
 '3.a.1',
 '3.b.1',
 '3.b.2',
 '3.b.3',
 '3.c.1',
 '3.d.1']
train['none'] = 1-train[label_cols].max(axis=1)

In [ ]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

def tokenize(s): return re_tok.sub(r' \1 ', s).split()

n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [ ]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

x = trn_term_doc
test_x = test_term_doc

In [ ]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

preds = np.zeros((len(test), len(label_cols)))

In [ ]:
for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
   # preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
   # int_prediction = (preds).astype(int)

In [ ]:
preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    